# Documentación de Scrapers

Este notebook contiene la documentación y ejemplos de uso de los componentes de scraping y construcción de modelos de datos disponibles en la carpeta `src/scrapers`.

## Índice

1. [Scraping de Páginas (Deep Scraper)](#scraping-de-paginas)
   - Scraping de una sola página (`page_scraper`)
   - Crawling recursivo (`page_deep_scraper`)
2. [Construcción de Modelos (Model Builder)](#construccion-de-modelos)
   - Identidad de la compañía (`data_to_identity`)
   - Generación del modelo completo (`from_url_model`)
3. [Utilidades de Extracción](#utilidades-de-extraccion)
   - Overview de módulos de utilidad

---

*Nota: Este documento debe actualizarse a medida que se agreguen nuevas funcionalidades al módulo de scrapers.*

## ⚙️ Configuración Notebook
Define el PATH para el correcto funcionamiento del notebook y la importación de módulos.

In [1]:
# Configurar el PYTHONPATH para que Python encuentre los módulos
import sys
from pathlib import Path
import os

# Obtener la ruta del directorio src de forma robusta
current_dir = Path(os.getcwd()).resolve()

# Buscar el directorio raíz que contiene 'scrapers'
src_path = None
search_dir = current_dir

# Buscar hasta 3 niveles arriba
for _ in range(3):
    if (search_dir / 'scrapers').exists():
        src_path = search_dir
        break
    search_dir = search_dir.parent

# Si no se encontró, usar el directorio actual si estamos dentro de scr
if src_path is None:
    if current_dir.name == 'src':
        src_path = current_dir
    else:
        # Asumir que estamos en src/scrapers y subir un nivel
        src_path = current_dir.parent if current_dir.name == 'scrapers' else current_dir

# Agregar src al PYTHONPATH si no está ya
if src_path and str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"📁 Agregado al PYTHONPATH: {src_path}")


📁 Agregado al PYTHONPATH: C:\Users\Usuario\Desktop\data_scraper\src


---
## 1. Scraping de Páginas (Deep Scraper) <a name="scraping-de-paginas"></a>

El módulo `page_deep_scraper.py` se encarga de la extracción de contenido web. Funciona en dos niveles:
1. **Nivel Página**: Extrae links y textos de una URL específica, segmentando por `head`, `header`, `main` y `footer`.
2. **Nivel Sitio (Crawler)**: Navega recursivamente por un dominio (BFS) para mapear su estructura.

In [2]:
# Importar funciones de scraping
from scrapers.page_deep_scraper import page_scraper, page_deep_scraper
import json

# Función auxiliar para imprimir JSON bonito
def print_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

### `page_scraper(url_base)`

Extrae contenido de **una sola URL**.

**Retorna:** Un objeto de página con:
- `page`: La URL procesada.
- `links`: Diccionario con listas de URLs encontradas en `head`, `header`, `main`, `footer`.
- `texts`: Diccionario con listas de textos encontrados en las mismas secciones.

In [4]:
# Ejemplo: Scrapear una sola página
TEST_URL = "https://fraccional.cl"

try:
    page_data = await page_scraper(TEST_URL)
    
    print(f"✅ Página scrapeada: {page_data['page']}")
    print("\nSecciones con links:")
    for section, links in page_data['links'].items():
        print(f"  - {section}: {len(links)} links")
        
    print("\nSecciones con textos:")
    for section, texts in page_data['texts'].items():
        print(f"  - {section}: {len(texts)} textos")
        
except Exception as e:
    print(f"❌ Error: {e}")

✅ Página scrapeada: https://fraccional.cl

Secciones con links:
  - head: 1 links
  - header: 9 links
  - main: 159 links
  - footer: 0 links

Secciones con textos:
  - head: 0 textos
  - header: 17 textos
  - main: 544 textos
  - footer: 4 textos


### `page_deep_scraper(url_base, max_pages=100)`

Realiza un **crawl recursivo** (Breadth-First Search) comenzando desde `url_base`.

**Características:**
- Respeta el **dominio raíz** (no sale del sitio).
- Detecta y evita ciclos (visita cada URL una sola vez).
- Limita la cantidad de páginas visitadas con `max_pages`.
- Captura errores por página sin detener el proceso global.

**Retorna:** Un objeto resumen con:
- `rootDomain`: El dominio base.
- `pagesScraped`: Cantidad total de páginas procesadas.
- `allInternalLinks`: Todos los links internos únicos encontrados.
- `pages`: Diccionario con el detalle de cada página (resultado de `page_scraper`).

In [6]:
# Ejemplo: Deep Scraper (limitado a 3 páginas para demo)
try:
    deep_data = await page_deep_scraper(TEST_URL, max_pages=50)
    
    print(f"🌎 Root Domain: {deep_data['rootDomain']}")
    print(f"📄 Páginas scrapeadas: {deep_data['pagesScraped']}")
    print(f"🔗 Total links internos únicos: {len(deep_data['allInternalLinks'])}")
    print("\nURLs visitadas:")
    for url in deep_data['pages'].keys():
        print(f"  - {url}")
        
except Exception as e:
    print(f"❌ Error: {e}")

🌎 Root Domain: fraccional.cl
📄 Páginas scrapeadas: 50
🔗 Total links internos únicos: 416

URLs visitadas:
  - https://fraccional.cl
  - https://fraccional.cl/proyectos?address_level3=%C3%91u%C3%B1oa
  - https://fraccional.cl/gestores/zuba-inversiones
  - https://fraccional.cl/proyectos?address_level3=Asunci%C3%B3n
  - https://fraccional.cl/blog/posts/como-gana-plata-fraccional-2
  - https://fraccional.cl/blog/posts/el-secreto-detras-de-su-ascenso-al-top-1-de-la-riqueza-mundial
  - https://forms.fraccional.cl/geniobot
  - https://fraccional.cl/blog/posts/la-forma-sigue-las-finanzas
  - https://fraccional.cl/blog/posts/como-invertir-en-btc
  - https://fraccional.cl/blog/posts/que-debes-declarar-si-compraste-en-fraccional
  - https://fraccional.cl/blog/posts/alternativas-de-inversiones
  - https://fraccional.cl/pt-BR
  - https://fraccional.cl/proyectos/unidades/paraguay-zuba-13/235
  - https://fraccional.cl/blog/posts/como-china-destruyo-su-mercado-inmobiliario-en-tiempo-record
  - https:

---
## 2. Construcción de Modelos (Model Builder) <a name="construccion-de-modelos"></a>

El módulo `model_builder.py` transforma la data "cruda" del scraper en un modelo estructurado (Company Model) listo para ser almacenado en la base de datos.

In [8]:
from scrapers.model_builder import data_to_identity, from_url_model, page_deep_scraped_to_dataSources

### `data_to_identity(url, name, slug, primary_domain)`

Normaliza y genera la identidad básica de una compañía (slug, name, primary_domain).

**Reglas:**
- `primaryDomain`: Se normaliza usando `tldextract` y se limpia de `www.`.
- `slug`: Se genera a partir del dominio si no se provee. Se reemplazan espacios por guiones.
- `name`: Se genera a partir del slug si no se provee (Capitalizado).

In [9]:
identity = data_to_identity(
    url="https://www.ejemplo.negocio.com/inicio",
    name=None, # Auto-generar
    slug=None  # Auto-generar
)

print("🆔 Identidad Generada:")
print_json(identity)

🆔 Identidad Generada:
{
  "slug": "negocio",
  "name": "Negocio",
  "primaryDomain": "negocio.com"
}


### `from_url_model(url, name, slug, primary_domain)`

Es la función de alto nivel que orquesta todo el proceso:
1. Genera la identidad.
2. Ejecuta `page_deep_scraper`.
3. Convierte el resultado en una lista de `dataSources` estandarizada.
4. Retorna el objeto completo de la compañía.

**Estructura del Output:**
```json
{
  "slug": "...",
  "name": "...",
  "primaryDomain": "...",
  "dataSources": [
     { "url": "...", "links": {...}, "texts": {...} },
     ...
  ]
}
```

In [10]:
# Ejemplo: Crear modelo completo desde una URL (con límite para demo)
# Nota: from_url_model llama a page_deep_scraper internamente.
# Para este ejemplo, simularemos una llamada rápida.

print("🚀 Iniciando construcción del modelo (esto puede tardar unos segundos)...")

try:
    company_model = await from_url_model(url=TEST_URL)
    
    print("✅ Modelo construido exitosamente:")
    print(f"  Nombre: {company_model['name']}")
    print(f"  Slug: {company_model['slug']}")
    print(f"  DataSources: {len(company_model['dataSources'])}")
    
    # Ver un dataSource de ejemplo
    if company_model['dataSources']:
        print("\nEjemplo de DataSource[0]:")
        first_ds = company_model['dataSources'][0]
        print(f"  URL: {first_ds['url']}")
        print(f"  Links Head: {len(first_ds.get('links', {}).get('head', []))}")

except Exception as e:
    print(f"❌ Error: {e}")

🚀 Iniciando construcción del modelo (esto puede tardar unos segundos)...
✅ Modelo construido exitosamente:
  Nombre: Fraccional
  Slug: fraccional
  DataSources: 587

Ejemplo de DataSource[0]:
  URL: https://fraccional.cl
  Links Head: 1


---
## 3. Utilidades de Extracción <a name="utilidades-de-extraccion"></a>

La carpeta `src/scrapers/utils` contiene herramientas de bajo nivel usadas por los scrapers. Estas pueden ser usadas independientemente para tareas específicas o debugging.

### Componentes Principales:

- **`requestHTTP.py`**: Manejo de peticiones HTTP con `aiohttp` y `playwright` (en el futuro).
- **`html_spliter...py`**: Divide el HTML crudo en secciones semánticas (`head`, `header`, `main`, `footer`).
- **`urls_extractor...py`**: Extrae todas las URLs crudas de un string HTML.
- **`urls_utilities_cleaner.py`**: Limpia URLs (quita espacios, caracteres inválidos).
- **`text_extractor...py`**: Extrae texto visible limpio del HTML.


In [11]:
# Ejemplo: Uso directo de utilidades para debug
from scrapers.utils.requestHTTP import fetch_html
from scrapers.utils.html_spliter_head_header_main_footer import html_spliter_head_header_main_footer

try:
    # 1. Fetch HTML manual
    raw_html = await fetch_html(TEST_URL)
    print(f"HTML descargado: {len(raw_html)} caracteres")
    
    # 2. Split HTML
    sections = html_spliter_head_header_main_footer(raw_html)
    print("Tamaño de secciones:")
    for sec, content in sections.items():
        print(f"  {sec}: {len(content)} caracteres")
        
except Exception as e:
    print(f"Error: {e}")

HTML descargado: 1712074 caracteres
Tamaño de secciones:
  head: 11592 caracteres
  header: 40032 caracteres
  main: 1352639 caracteres
  footer: 1522 caracteres


---
## Extensibilidad

Para agregar nuevos scrapers o funcionalidades:

1. **Nuevos Parsers**: Agregarlos en `src/scrapers/utils/` si son utilidades genéricas de limpieza o extracción.
2. **Lógica de Scraping**: Modificar `src/scrapers/page_deep_scraper.py` para mejorar la navegación o la extracción de datos específica.
3. **Modelado**: Actualizar `src/scrapers/model_builder.py` si cambia la estructura de la base de datos (DataSources).

Consulte `src/scrapers/estructura_recomendada.txt` para ver la estructura modular planificada para el futuro.